In [ ]:
%matplotlib inline
import os
os.environ['PY3_PROD'] = '1'
%load_ext autoreload
%autoreload 2
os.system('kinit')

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import copy
import numpy as np
import math
import matplotlib as mpl
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import datetime
import itertools
import pandas as pd

from pycmqlib3.utility import dbaccess, dataseries, misc
import pycmqlib3.analytics.data_handler as dh


import sys
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

sys.path.append("C:/dev/pyktrader3/")
sys.path.append("C:/dev/wtpy/")
sys.path.append("C:/dev/akshare/")
sys.path.append("C:/dev/wtdev/")

from pycmqlib3.analytics.tstool import *
from pycmqlib3.analytics.btmetrics import *
from pycmqlib3.analytics.backtest_utils import *
from misc_scripts.fun_factor_update import *
from pycmqlib3.strategy.signal_repo import *
from bktest.backtest_grid_search import *

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
mpl.rcParams['figure.figsize'] = (12, 8)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))
lead_lag_config = {
    'll_left': -20,
    'll_right': 120,
    'll_spacing': 5,
    'll_sub_win': [(datetime.date(2008, 1, 1), datetime.date(2016, 12, 31)), 
                   (datetime.date(2017, 1, 1), datetime.date(2022, 12, 31)),],
}

ll_keys = ['fullsample'] + ['%s:%s' % (sd.strftime('%Y-%b-%d'), ed.strftime('%Y-%b-%d')) for sd, ed in lead_lag_config['ll_sub_win']]


# product group and starting date

In [ ]:
from misc_scripts.update_fut_prices import load_saved_fut
tday = datetime.date(2024,11,2)

df = load_saved_fut(tday, freq='d')
spot_df = load_fun_data(tday)

start_date = df.index[0]
end_date = tday

cdates = pd.date_range(start=start_date, end=tday, freq='D')
bdates = pd.bdate_range(start=start_date, end=end_date, freq='C', holidays=misc.CHN_Holidays)


In [ ]:
bdates = [d for d in bdates if d.date() not in [datetime.date(2014,1,2), datetime.date(2014,1,3)]]
df = df.reindex(index=bdates)

In [ ]:
for asset_cont in df.columns.get_level_values(0).unique():    
    flag = df[(asset_cont, "a1505")].isna()
    df.loc[flag, (asset_cont, 'a1505')]= df.loc[flag, (asset_cont, 'close')]            
    for traded_price in ['n305', 'n310', 'n315', 'n450']:
        flag = df[(asset_cont, traded_price)].isna()
        df.loc[flag, (asset_cont, traded_price)]= df.loc[flag, (asset_cont, 'a1505')]
        

In [ ]:
asset_pairs = [('j', 'i'), ('j', 'jm'), ('rb', 'i'), ('hc', 'i'),]
beta_ret_dict = {}
beta_dict = {}
beta_win = 122
vwin = 20
for trade_asset, index_asset in asset_pairs:
    asset_df = df[[(f'{index_asset}c1', 'close'), (f'{trade_asset}c1', 'close')]].copy(deep=True).droplevel([1], axis=1)
    asset_df = asset_df.pct_change().dropna()
    asset_df.columns = [col[:-2] for col in asset_df.columns]    
    for asset in asset_df:
        asset_df[f'{asset}_pct'] = asset_df[asset].rolling(5).mean()
        asset_df[f'{asset}_vol'] = asset_df[asset].rolling(vwin).std()
    asset_df['beta'] = asset_df[f'{index_asset}_pct'].rolling(beta_win).cov(asset_df[f'{trade_asset}_pct'])/asset_df[f'{index_asset}_pct'].rolling(beta_win).var()
    key = '_'.join([trade_asset, index_asset])
    asset_df['pct_chg'] = asset_df[trade_asset] - asset_df['beta'] * asset_df[index_asset].fillna(0)
    asset_df['pct_vol'] = asset_df['pct_chg'].rolling(vwin).std()
    asset_df['trade_leg'] = asset_df[f'{trade_asset}_vol']/asset_df['pct_vol']
    asset_df['index_leg'] = - asset_df[f'{index_asset}_vol'] / asset_df['pct_vol'] * asset_df['beta']
    
    beta_ret_dict[key] = asset_df['pct_chg']
    beta_dict[key] = asset_df[['beta', 'trade_leg', 'index_leg']]
    

In [ ]:
vol_win = 20
product_list = list(set([col[:-2] for col in df.columns.get_level_values(0).unique()]))

for asset in product_list:    
    if (asset+'c2', 'close') not in df.columns:
        print(asset)
        continue
    spot_df[f'{asset}_ryield'] = ((
        np.log(df[(asset+'c1', 'close')]) - np.log(df[(asset+'c2', 'close')]) - 
        df[(asset+'c1', 'shift')] + df[(asset+'c2', 'shift')])/(
        df[(asset+'c2', 'expiry')] - df[(asset+'c1', 'expiry')]).dt.days*365.0 + spot_df['r007_cn'].ffill()/100).dropna()
    spot_df[f'{asset}_px'] = df[(asset+'c1', 'close')]
    spot_df[f'{asset}_logret'] = np.log(df[(asset+'c1', 'close')]).dropna().diff()
    spot_df[f'{asset}_pctchg'] = np.log(df[(asset+'c1', 'close')]).dropna().pct_change()    
    spot_df[f'{asset}_logret2'] = np.log(df[(asset+'c2', 'close')]).dropna().diff()
    spot_df[f'{asset}_basmom'] =spot_df[f'{asset}_logret'] - spot_df[f'{asset}_logret2']        
    spot_df[f'{asset}_pctvol'] = spot_df[f'{asset}_pctchg'].dropna().rolling(vol_win).std()
    spot_df[f'{asset}_basmom5'] = spot_df[f'{asset}_basmom'].dropna().rolling(5).sum()
    spot_df[f'{asset}_basmom10'] = spot_df[f'{asset}_basmom'].dropna().rolling(10).sum()
    spot_df[f'{asset}_basmom20'] = spot_df[f'{asset}_basmom'].dropna().rolling(20).sum()
    spot_df[f'{asset}_basmom40'] = spot_df[f'{asset}_basmom'].dropna().rolling(40).sum()
    spot_df[f'{asset}_basmom60'] = spot_df[f'{asset}_basmom'].dropna().rolling(60).sum()
    spot_df[f'{asset}_basmom100'] = spot_df[f'{asset}_basmom'].dropna().rolling(100).sum()
    spot_df[f'{asset}_basmom120'] = spot_df[f'{asset}_basmom'].dropna().rolling(120).sum()
    spot_df[f'{asset}_basmom170'] = spot_df[f'{asset}_basmom'].dropna().rolling(170).sum()
    spot_df[f'{asset}_basmom180'] = spot_df[f'{asset}_basmom'].dropna().rolling(180).sum()
    
spot_df['hc_rb_diff'] = np.log(df[('hcc1', 'close')]) - np.log(df[('rbc1', 'close')])

In [ ]:
from pycmqlib3.strategy.signal_repo import *
from misc_scripts.fun_factor_update import *


In [ ]:
broad_mkts = [
    'rb', 'hc', 'i', 'j', 'jm', 'FG', 'v', 'SM', 'SF', 'SA',
    'cu', 'al', 'zn', 'ni', 'pb', 'sn', 'ss', 'si', 'ao', 'au', 'ag',#'bc'
    'ru', 'l', 'pp', 'TA', 'MA', 'sc', 'eb', 'eg', 'UR', 'lu', #'bu', 'fu',
    'm', 'RM', 'y', 'p', 'OI', 'a', 'c', 'CF', 'jd', 'AP', 'lh', #'cs',
]

ind_mkts = ['rb', 'hc', 'i', 'j', 'jm', 'FG', 'v', 'SA', 'UR', 'SM', 'SF', 
    'cu', 'al', 'zn', 'ni', 'pb', 'sn', 'ss', 'ao', 'au', 'ag',#'bc', 'si', 
    'ru', 'l', 'pp', 'TA', 'MA', 'sc', 'eb', 'eg', 'lu', #'bu', 'fu',
]

signal_by_func = {
    **factors_by_func,
    **{ 
    }
}

signal_by_spread = {
    **factors_by_spread,
    **{
        
    }
}

signal_by_beta_neutral = {
    **factors_by_beta_neutral,
    **{
        
    }
}

signal_by_asset = {
    **factors_by_asset,
    **{
        "ryield_ema": broad_mkts,
        'metal_pbc_ema': ['cu', 'al', 'zn', 'pb', 'ni', 'ss', 'sn', 'ao',
                          'rb', 'hc', 'i', 'v', 'FG', 'SA'],
        'metal_mom_hlrhys': ['cu', 'al', 'zn', 'pb', 'ni', 'ss', 'sn', 'ao', 
                             'rb', 'hc', 'i', 'j', 'jm', 'SM', 'SF', 'v', 'FG', 'SA'],
        'metal_inv_hlr': ['cu', 'al', 'zn', 'pb', 'ni', 'ss', 'sn', 'ao',
                          'rb', 'hc', 'i', 'v', 'FG', 'SA'],
        'metal_inv_lyoy_hlr': ['cu', 'al', 'zn', 'pb', 'ni', 'ss', 'sn', 'ao', 
                               'rb', 'hc', 'i', 'v', 'FG', 'SA'], 
        
        'lme_base_ts_mds': ['cu', 'al', 'zn', 'pb', 'ni', 'sn'],
        'lme_base_ts_hlr': ['cu', 'al', 'zn', 'pb', 'ni', 'sn'],
        'lme_futbasis_ma': ['cu', 'al', 'zn', 'pb', 'ni', 'sn'],
        'base_inv_shfe_ma': ['cu', 'al', 'zn', 'pb', 'ni', 'sn'],
        'base_inv_lme_ma': ['cu', 'al', 'zn', 'pb', 'ni', 'sn'],
        'base_inv_exch_ma': ['cu', 'al', 'zn', 'pb', 'ni', 'sn'],
        'base_phybas_carry_ma': ['cu', 'al', 'zn', 'ni', 'sn', 'pb'],
        'base_inv_mds': ['cu', 'al', 'zn', 'ni', 'sn', 'pb', 'ss', 'si', 'ao'],
        'base_tc_1y_zs': ['cu', 'pb', 'zn'],
        'base_tc_2y_zs': ['cu', 'pb', 'sn'],
        'base_cifprem_1y_zs': ['cu', 'al', 'zn', 'ni'],
        'base_phybasmom_1m_zs': ['cu', 'al', 'zn', 'ni', 'pb', 'sn'],
        'base_phybasmom_1y_zs': ['cu', 'al', 'zn', 'ni', 'pb', 'sn'],
    }
}

signal_by_data = {
    **single_factors,
    **{
        'steel_margin_lvl_fast': ['rb', 'hc', 'i', 'j', 'jm'],
        'strip_hsec_lvl_mid': ['rb', 'hc', 'i', 'j', 'jm'],
        'io_removal_lvl': ['i'],
        'io_removal_lyoy': ['i'],
        'io_millinv_lyoy': ['hc', 'i'],
        'io_inv_rmv_ratio_1y': ['i'],
        'ioarb_px_hlr': ['rb', 'hc', 'i'],
        'ioarb_px_hlrhys': ['rb', 'hc', 'i'],
        'steel_sinv_lyoy_zs': ['rb', 'hc', 'i', 'FG', 'v'],
        'steel_sinv_lyoy_mds': ['rb', 'hc', 'i', 'FG', 'v'],
        'rbsales_lyoy_mom_lt': ['rb'],
        'rb_sales_inv_ratio_lyoy': ['rb'],
        'fef_c1_c2_ratio_or_qtl': ['rb', 'hc', 'j'],
        'fef_fly_ratio_or_qtl': ['rb', 'hc', 'j'],
        'fef_basmom_or_qtl': ['rb', 'hc'],
        'fef_basmom5_or_qtl': ['rb', 'hc'],      
        "const_etf_mom_zsa": ["rb", "i", "j", "FG", "v"],
        "const_etf_mom_ewm": ["rb", "i", "j", "FG", "v"],
        "prop_etf_mom_dbth_zs": ["rb", "i", "FG", "v"],
        "prop_etf_mom_dbth_qtl": ["rb", "i", "FG", "v"],
        "prop_etf_mom_dbth_qtl2": ["rb", "i", "FG", "v"],  
        
        "al_alumina_qtl": ['al'],
        "al_alumina_yoy_qtl": ['al'],
        "al_coal_qtl": ['al'],
        "ni_nis_mom_qtl": ['ni'],
        "ni_ore_qtl": ['ni'],
        "sn_conc_spot_hlr": ['sn'],
        'cu_prem_usd_zsa': ['cu'],
        'cu_prem_usd_md': ['cu'],
        'cu_phybasis_zsa': ['cu'],
        'cu_phybasis_hlr': ['cu'],
        "base_etf_mom_zsa": ["cu", "al", "zn", "pb", "ni", "sn"],
        "base_etf_mom_ewm": ["cu", "al", "zn", "pb", "ni", "sn"],        
    }
}

feature_setup = {
    **signal_store,
    **{      
    }
}

# strategy portfolio config

In [ ]:
prem_fact = 25000
metal_fact = 8000
misc_fact = 12000
ferr_fact = 20000
base_fact = 20000

prem_strats = [
    ["ryield_ema", 0.082*prem_fact], 
    ["ryield_ema_xdemean", 0.20*prem_fact], 
    ["ryield_st_zsa", 0.016*prem_fact],
    ["ryield_st_zsa_xdemean", 0.02*prem_fact],
    ["ryield_lt_zsa", 0.025*prem_fact],
    ["ryield_lt_zsa_xdemean", 0.045*prem_fact],
    
    ["basmom20_ema", 0.013*prem_fact],
    ["basmom20_ema_xdemean", 0.032*prem_fact],           
    ["basmom60_ema", 0.022*prem_fact],
    ["basmom60_ema_xdemean", 0.027*prem_fact], 
    ["basmom120_ema", 0.02*prem_fact],
    ["basmom120_ema_xdemean", 0.035*prem_fact],

    ["mom_momma240", 0.02*prem_fact],
    ["mom_momma240_xdemean", 0.062*prem_fact],
    ["mom_momma20", 0.009*prem_fact],
    ["mom_momma20_xdemean", 0.0105*prem_fact],     
    ["mom_hlr_st", 0.062*prem_fact],
    ["mom_hlr_st_xdemean", 0.04*prem_fact], 
    ["mom_hlr_lt_xdemean", 0.014*prem_fact],
    ["mom_hlr_yr_xdemean", 0.00*prem_fact],   
]

metal_strats = [
    ['metal_pbc_ema', 0.457*metal_fact],
    ['metal_pbc_ema_xdemean', 0.315*metal_fact],    
    ['metal_mom_hlrhys', 0.316*metal_fact],
    ['metal_mom_hlrhys_xdemean', 0.042*metal_fact],
    
    ['metal_inv_hlr', 0.093*metal_fact],
    ['metal_inv_hlr_xdemean', 2.26*metal_fact],
#     ['metal_inv_lyoy_hlr', 0],        
]

misc_strats = [
    ["exch_wnt_hlr", 0.336*misc_fact],
    ["exch_wnt_hlr_xdemean", 0.336*misc_fact],
    ["exch_wnt_yoy_hlr", 0.336*misc_fact],
    ["exch_wnt_yoy_hlr_xdemean", 0.336*misc_fact],        
    ["leadlag_d_mid", 0.28*misc_fact],
    ['leadlag2_mr_d', 0.47*misc_fact],
    ['pair_mr_1y', 1.4*misc_fact],
    ["hc_rb_diff_20", 0.1*misc_fact],    
]

ferrous_strats = [
    ['io_inv_rmv_ratio_1y', 1.71*ferr_fact],
    ['io_removal_lvl', 1.04*ferr_fact],
    ['io_removal_lyoy', 0.1*ferr_fact],
    ['io_millinv_lyoy', 0.74*ferr_fact],
    ['strip_hsec_lvl_mid', 0.05*ferr_fact],
    ['steel_margin_lvl_fast', 0.195*ferr_fact],
    ["steel_sinv_lyoy_zs", 0.2*ferr_fact],
    ["steel_sinv_lyoy_mds", 0.1*ferr_fact],
    ['fef_c1_c2_ratio_or_qtl', 0.07*ferr_fact],    
    ['fef_fly_ratio_or_qtl', 0.155*ferr_fact],
    ['fef_basmom_or_qtl', 0*ferr_fact],
    ['fef_basmom5_or_qtl', 0.39*ferr_fact],
    ['fef_c1_c2_ratio_spd_qtl', 0.25*ferr_fact],    
    ['fef_basmom5_spd_qtl', 0.51*ferr_fact],
    
    ['io_pinv31_lvl_zsa', 0*ferr_fact],
    ['io_pinv45_lvl_hlr', 0.217*ferr_fact],
    ['ioarb_px_hlr', 0.03*ferr_fact],
    ['ioarb_px_hlrhys', 0.03*ferr_fact],
    
    ['ioarb_spd_qtl_1y', 0.62*ferr_fact],
    ['const_etf_mom_zsa', 0*ferr_fact],
    ['const_etf_mom_ewm', 0.164*ferr_fact],
    ['prop_etf_mom_dbth_zs', 0.178*ferr_fact],
    ['prop_etf_mom_dbth_qtl', 0.076*ferr_fact],
    ['rbsales_lyoy_mom_lt', 0.2*ferr_fact],    
    ['rb_sales_inv_ratio_lyoy', 0.2*ferr_fact],
    ['rbsales_lyoy_spd_st', 0.5*ferr_fact],
    ['rbhc_dmd_mds', 0.3*ferr_fact],
    ['rbhc_sinv_mds', 0.2*ferr_fact],        
]

base_strats = [
    ['lme_base_ts_mds', 0.29*base_fact],
    ['lme_base_ts_mds_xdemean', 0.24*base_fact],
    ['lme_base_ts_hlr', 0*base_fact],
    ['lme_base_ts_hlr_xdemean', 0.035*base_fact],
    ['lme_futbasis_ma', 0.016*base_fact],
    ['lme_futbasis_ma_xdemean', 0.4*base_fact],     
    ['base_phybas_carry_ma', 0.354*base_fact],
    ['base_phybas_carry_ma_xdemean', 0*base_fact],   
    ['base_phybasmom_1m_zs', 0.05*base_fact],
    ['base_phybasmom_1m_zs_xdemean', 0.47*base_fact],         
    ['base_phybasmom_1y_zs', 0.028*base_fact],
    ['base_phybasmom_1y_zs_xdemean', 0*base_fact],          
    ['base_cifprem_1y_zs', 0.42*base_fact],
    ['base_cifprem_1y_zs_xdemean', 0.006*base_fact],       
    ['base_tc_1y_zs', 0.53*base_fact],
    ['base_tc_2y_zs', 0.215*base_fact],     
    ['base_inv_mds', 0.19*base_fact],
    ['base_inv_mds_xdemean', 0.31*base_fact],         
    ['base_inv_exch_ma', 0.006*base_fact],
    ['base_inv_exch_ma_xdemean', 0.164*base_fact],     
#     ['base_inv_shfe_ma', 1.0],
#     ['base_inv_shfe_ma_xdemean', 1.0],      
    ['base_etf_mom_zsa', 0.2*base_fact],
    ['base_etf_mom_ewm', 0.0*base_fact],           
    ['al_alumina_qtl', 0.038*base_fact],
    ['al_alumina_yoy_qtl', 0.106*base_fact],     
    ['al_coal_qtl', 0.176*base_fact],
    ['ni_nis_mom_qtl', 0.13*base_fact],      
    ['ni_ore_qtl', 0.14*base_fact],
    ['sn_conc_spot_hlr', 0.22*base_fact],      
]


strat_group = prem_strats + metal_strats + misc_strats + ferrous_strats + base_strats

# run backtest for a portfolio group

In [ ]:
bt_dict = {}
pnl_dict = {}
holding_dict = {}
signal_dict = {}

In [ ]:
file_folder = "C:\\dev\\data\\data_cache\\"

start_date = datetime.date(2008,1,1)
end_date = datetime.date.today()

portfolio_map = {
    'prod': strat_group,
}

risk_scaling = 1
asset_scaling = False
std_win = 20
trd_cost = 2e-4

pnl_tenors = ['6m', '1y', '2y', '3y', '4y', '5y', '6y', '7y', '8y', '9y', '10y', '12y', '15y']

asset_list = [
    'rb', 'hc', 'i', 'j', 'jm', 'FG', 'v', 'SM', 'SF', 'SA',
    'cu', 'al', 'zn', 'ni', 'pb', 'sn', 'ss', 'ao', 'au', 'ag', #'si', 
    'ru', 'l', 'pp', 'TA', 'MA', 'sc', 'eb', 'eg', 'UR', 'lu', 'bu', 'fu',
    'm', 'RM', 'y', 'p', 'OI', 'a', 'c', 'CF', 'jd', 'AP', 'lh', 'cs', #'bc'
]

product_list = [asset+'c1' for asset in asset_list]

close_prices = df.loc[:, df.columns.get_level_values(0).isin(product_list) &
                      (df.columns.get_level_values(1)=='close')].droplevel([1], axis=1).dropna(how='all').ffill()
close_prices.columns = [col[:-2] for col in close_prices.columns]
close_prices = close_prices[asset_list]
vol_df = close_prices.pct_change().rolling(std_win).std()

portfolio_name = 'prod'
run_key = portfolio_name
strat_portfolio = portfolio_map[portfolio_name]

pos_sum = pd.DataFrame(0, index=df.index, columns=asset_list)

bt_dict[run_key] = {}
signal_dict[run_key] = {}
pnl_dict[run_key] = {}
holding_dict[run_key] = {}

agg_signal = pd.DataFrame(index=df.index, columns=asset_list)
agg_pnl = pd.DataFrame(index=df.index, columns=asset_list)
pnl_by_signal = pd.DataFrame(index=df.index, columns=[signal_name for signal_name, weight in strat_portfolio])
port_start = pd.to_datetime('2010-01-01')

for signal_name, weight in strat_portfolio:    
    print(signal_name)
    if signal_name in signal_by_func:
        func = signal_by_func[signal_name]['func']
        func_args = signal_by_func[signal_name]['args']     
        signal_df = func(df, spot_df, **func_args)
    elif signal_name in signal_by_spread:
        signal_ts = get_funda_signal_from_store(spot_df, signal_name, price_df=df, signal_repo=feature_setup)
        signal_df = pd.DataFrame({asset: signal_ts * w for asset, w in signal_by_spread[signal_name]})            
    elif signal_name in signal_by_beta_neutral:        
        signal_ts = get_funda_signal_from_store(spot_df, signal_name, price_df=df, signal_repo=feature_setup)
        signal_df = pd.DataFrame(index=signal_ts.index)        
        for trade_asset, index_asset, w in signal_by_beta_neutral[signal_name]:
            if trade_asset not in signal_df.columns:
                signal_df[trade_asset] = 0
            if index_asset not in signal_df.columns:
                signal_df[index_asset] = 0
            sig_df = pd.concat([
                signal_ts.to_frame('signal'),
                beta_dict[key]['trade_leg'].to_frame('trade_w'), 
                beta_dict[key]['index_leg'].to_frame('index_w')], axis=1).ffill()
            sig_df[trade_asset] = sig_df['signal'] * sig_df['trade_w']
            sig_df[index_asset] = sig_df['signal'] * sig_df['index_w']
            signal_df[[trade_asset, index_asset]] += sig_df[[trade_asset, index_asset]]
    elif signal_name in signal_by_data:        
        signal_ts = get_funda_signal_from_store(spot_df, signal_name, price_df=df, signal_repo=feature_setup)
        signal_df = pd.DataFrame({asset: signal_ts for asset in signal_by_data[signal_name] if asset in asset_list})
    else:
        xs_type = signal_name.split("_")[-1]
        if xs_type in ['xdemean', 'xscore', 'xrank',]:
            sig_name = "_".join(signal_name.split("_")[:-1])
        else:
            sig_name = signal_name
            xs_type = 'ts'
        if sig_name in signal_by_asset:
            trade_asset = [asset for asset in asset_list if asset in signal_by_asset[sig_name]]
            signal_df = pd.DataFrame(index=df.index, columns=trade_asset)
            for asset in trade_asset:
                signal_df[asset] = get_funda_signal_from_store(spot_df, signal_name, price_df=df, asset=asset, signal_repo=feature_setup)
            if xs_type == 'xdemean':
                signal_df = xs_demean(signal_df)
            elif xs_type == 'xscore':
                signal_df = xs_score(signal_df)
        else:
            print("unsupported signal")

    signal_exec_param = signal_execution_config.get(signal_name, {"win": "n305", "lag": 1})
    traded_products = [col + 'c1' for col in signal_df.columns]
    
    if signal_name in signal_store:
        _, _, _, _, _, _, _, post_func, _, _ = signal_store[signal_name][1]
        last_func = post_func.split('|')[-1]
        if 'buf' in last_func:
            buffer_size = float(last_func[3:])
            signal_df = signal_buffer(signal_df, buffer_size)
        elif 'bfc' in last_func:
            buffer_size = float(last_func[3:])
            signal_df = signal_cost_optim(signal_df, buffer_size, vol_df[signal_df.columns],
                                          cost_dict=simple_cost(signal_df.columns, trd_cost=trd_cost),
                                          turnover_dict={},
                                          power=3)
    traded_prices = df.loc[:, df.columns.get_level_values(0).isin(traded_products) &
                      (df.columns.get_level_values(1)==signal_exec_param['win'])].droplevel([1], axis=1).dropna(how='all').ffill()
    traded_prices.columns = [col[:-2] for col in traded_prices.columns]
    traded_prices = traded_prices[signal_df.columns]    
    df_pxchg = traded_prices.pct_change()
    holding = generate_holding_from_signal(signal_df, vol_df[signal_df.columns], 
                                           risk_scaling=risk_scaling, 
                                           asset_scaling=asset_scaling)    
    bt_metrics = MetricsBase(holdings=holding[signal_df.columns],
                             returns=df_pxchg[signal_df.columns], 
                             shift_holdings=signal_exec_param["lag"],
                             cost_dict=simple_cost(holding.columns, trd_cost=trd_cost)
                            )
    bt_dict[run_key][signal_name] = bt_metrics
    signal_dict[run_key][signal_name] = signal_df
    holding_dict[run_key][signal_name] = holding
    
    pnl_stats = bt_metrics.calculate_pnl_stats(shift=0, tenors=pnl_tenors, perf_metrics=['sharpe', 'std', 'sortino', 'calmar'])
    perf_stats = transform_output(pnl_stats)    
    
    asset_pnl = bt_metrics.calculate_daily_pnl(traded_prices, close_prices[signal_df.columns], mode='ret')
    pnl_dict[run_key][signal_name] = asset_pnl
    print(f"\nsignal={signal_name}\n")
    display(perf_stats.round(2))
    iplot(asset_pnl.cumsum().sum(axis=1).to_frame("total")[port_start:], title=signal_name)
    iplot(asset_pnl.cumsum()[port_start:], title=signal_name)    
    display(pd.concat([pnl_stats['pnl_per_trade'].to_frame("bias"), pnl_stats['turnover'].to_frame("TO")], axis=1).round(2))
    display(pnl_stats['asset_sharpe_stats'].round(2))
    
    agg_pnl = agg_pnl.add((asset_pnl*weight).reindex_like(agg_pnl), fill_value=0)
    pnl_by_signal[signal_name] = (asset_pnl*weight).sum(axis=1)
    agg_signal = agg_signal.add((signal_df*weight).reindex_like(agg_signal), fill_value=0)

met_list = []
for asset in agg_pnl:
    sr_ts = calc_perf_by_tenors(agg_pnl[asset], tenors=pnl_tenors, metric='sharpe')
    met_list.append(sr_ts.to_frame(asset))
    
sr_df = pd.concat(met_list, axis=1)

port_pnl = agg_pnl.sum(axis=1)
met_list = []
for metric in ['sharpe', 'std']:
    sr_ts = calc_perf_by_tenors(port_pnl, tenors=pnl_tenors, metric=metric)
    sr_ts.index=pnl_tenors
    met_list.append(sr_ts.to_frame(metric))
port_df = pd.concat(met_list, axis=1)

display(port_df.round(3))
display(sr_df.round(3))    

iplot(agg_pnl.sum(axis=1).cumsum().to_frame('portfolio pnl'))
iplot(agg_pnl.cumsum(), title="portfolio pnl by asset")

# print(port_pnl[-40:])
# port_pnl.to_csv(file_folder + "port_pnl.csv")


In [ ]:
agg_signal

In [ ]:
prem_strats + metal_strats + misc_strats + ferrous_strats + base_strats

prem_strat_names = [elem[0] for elem in prem_strats]
metal_strat_names = [elem[0] for elem in metal_strats]
misc_strat_names = [elem[0] for elem in misc_strats]
ferrous_strat_names = [elem[0] for elem in ferrous_strats]
base_strat_names = [elem[0] for elem in base_strats]


In [ ]:
data_dict = {}
data_dict['prem_pnl'] = pnl_by_signal[prem_strat_names].sum(axis=1)
data_dict['misc_pnl'] = pnl_by_signal[misc_strat_names].sum(axis=1)
data_dict['metal_pnl'] = pnl_by_signal[metal_strat_names].sum(axis=1)
data_dict['ferrous_pnl'] = pnl_by_signal[ferrous_strat_names].sum(axis=1)
data_dict['base_pnl'] = pnl_by_signal[base_strat_names].sum(axis=1)

pnl_df = pd.DataFrame(data_dict)

corr_cutoff = '2017-01-01'
print("std:\n%s\n" % pnl_df[corr_cutoff:].std(axis=0))
print("std (total) = %s" % pnl_df[corr_cutoff:].sum(axis=1).std())
iplot(pnl_df.cumsum())

In [ ]:
import seaborn as sns
%matplotlib inline

corr_cutoff = '2017-01-01'
print("std:\n%s\n" % pnl_by_signal[corr_cutoff:].std(axis=0))
print("std (total) = %s" % pnl_by_signal[corr_cutoff:].sum(axis=1).std())
iplot(pnl_by_signal.cumsum())



In [ ]:
met_list = []
for sig in pnl_df:
    sr_ts = calc_perf_by_tenors(pnl_df[sig], tenors=pnl_tenors, metric='sharpe')
    met_list.append(sr_ts.to_frame(sig))
    
sr_df = pd.concat(met_list, axis=1)
sr_df

In [ ]:
pnl_w_df = pnl_df.resample('W').sum()
corr_df = pnl_w_df[corr_cutoff:].corr()

fig, ax = plt.subplots(figsize=(20,20))
sns.heatmap(corr_df.astype('float'), 
            #xticklabels=corr_df.columns, 
            #yticklabels=corr_df.columns, 
            #vmin=-1, vmax=1, 
            cmap='coolwarm',
            linewidth=.2,
            annot=True,
            ax=ax
           )


In [ ]:
pnl_df = pnl_by_signal.copy()
pnl_df['exch_wnt'] = pnl_df[[col for col in pnl_df if 'exch_wnt' in col]].sum(axis=1)
pnl_df = pnl_df[['exch_wnt', 'leadlag_d_mid', 'leadlag2_mr_d', 'pair_mr_1y', 'hc_rb_diff_20']]
# pnl_df['carry'] = pnl_df[['ryield_ema', 'ryield_ema_xdemean']].sum(axis=1)
# pnl_df['carrymom'] = pnl_df[["ryield_st_zsa", "ryield_st_zsa_xdemean", "ryield_lt_zsa", "ryield_lt_zsa_xdemean", "basmom20_ema", "basmom20_ema_xdemean"]].sum(axis=1)
# pnl_df['basmom'] = pnl_df[["basmom60_ema", "basmom60_ema_xdemean", "basmom120_ema", "basmom120_ema_xdemean"]].sum(axis=1)
# pnl_df['metal_pbc'] = pnl_df[['metal_pbc_ema', 'metal_pbc_ema_xdemean']].sum(axis=1)
# pnl_df['metal_mom'] = pnl_df[['metal_mom_hlrhys', 'metal_mom_hlrhys_xdemean']].sum(axis=1)
#pnl_df = pnl_df[['carry', 'carrymom', 'basmom', 'metal_pbc', 'metal_mom', 'leadlag_d_mid']]

In [ ]:
strat_std

In [ ]:
import pypfopt
from pypfopt import plotting
from pypfopt import risk_models
from pypfopt import EfficientFrontier
from pypfopt import expected_returns
#   
#pnl_df = pnl_df.copy()
dpnl = pnl_df[[col for col in pnl_df.columns]]['2017-01-01':]

if 'combo' in dpnl.columns:
    dpnl = dpnl.drop(columns=['combo'])
dpnl = dpnl.fillna(0)

strat_std = dpnl.std()

dpnl = dpnl/dpnl.std()

#risk_models.sample_cov(dpnl, returns_data=True, frequency=244)

max_sharpe = False

if max_sharpe:
    mu = expected_returns.mean_historical_return(dpnl, returns_data=True, frequency=244, compounding=False)
else:
    mu = None

S = risk_models.CovarianceShrinkage(dpnl, returns_data=True, frequency=244).ledoit_wolf()
ef = EfficientFrontier(mu, S, weight_bounds= (0, 1)) 

fact_idx = {}
for fact in dpnl.columns:
    fact_idx[fact] = ef.tickers.index(fact)

#ef.add_constraint(lambda w: w[fact_idx['ryield_ema_ts']] + w[fact_idx['ryield_ema_xdemean']] >= 0.3)
#ef.add_constraint(lambda w: w[fact_idx['ryield_ema_ts']] + w[fact_idx['ryield_ema_xdemean']] <= 0.4)
#ef.add_constraint(lambda w: w[fact_idx['ryield_ema_ts']] + w[fact_idx['ryield_ema_xdemean']] >= 0.3)
# ef.add_constraint(lambda w: w[fact_idx['tsmom']] + w[fact_idx['macro2']] <= 0.24)

#ef = EfficientFrontier(mu, S, weight_bounds=(0,1))
if max_sharpe:
    port_weights = ef.max_sharpe()
else:
    port_weights = ef.min_volatility()

#port_weights = ef.clean_weights()
port_weights = pd.Series(port_weights)
print("port_weights=\n%s\n" % port_weights)
final_weights = port_weights.div(strat_std)
print("final_weights=\n%s\n" % final_weights)
# mu = expected_returns.mean_historical_return(df)
# S = risk_models.sample_cov(df)

# # Optimize for maximal Sharpe ratio
# ef = EfficientFrontier(mu, S)
# weights = ef.max_sharpe()
# ef.portfolio_performance(verbose=True)

In [ ]:
import pickle

port_data = {'empirics': bt_empirics, 'scen_stats': scen_stats, 'scen_names': scen_names, 'holdings': holdings}

out_file = open('port_data_test.pkl','wb')

pickle.dump(port_data, out_file)

out_file.close()

In [ ]:
#pnl_stats = bt_empirics[run_key].calculate_pnl_stats(shift=0, tenors=pnl_tenors, perf_metrics=['sharpe', 'std', 'sortino', 'calmar'])

#pnl_stats['asset_sharpe_stats']

for asset in port_stats['asset_cumpnl'].columns:
    port_stats['asset_cumpnl'][asset]['2017-01-01':].plot(title=asset)
    plt.grid()
    plt.show()

In [ ]:
pos_sum[product_list]

In [ ]:
port_stats = bt_met.calculate_pnl_stats(shift=0, tenors=pnl_tenors)


In [ ]:
ts1 = bt_empirics['hot_elite'].calculate_pnl_stats(shift=0, tenors=pnl_tenors)['portfolio_cumpnl']
ts2 = bt_empirics['hot_test'].calculate_pnl_stats(shift=0, tenors=pnl_tenors)['portfolio_cumpnl']


# holding and std for strategy group

In [ ]:
file_folder = "C:\\dev\\data\\data_cache\\"

#port_stats['portfolio_cumpnl'].to_csv(file_folder + "port_pnl_est.csv")

print(bt_met.holdings)

print(port_stats['portfolio_pnl'].resample('M').sum()[-60:])


In [ ]:
tenors = ['1y', '2y', '3y', '5y', '7y', '9y', '11y']
data_list = []
run_key = 'hot_funda'
for scen_name, scen, pl_stat in zip(scen_names[run_key], scenarios, scen_stats[run_key]):
    weight = scen[2]
    data = [scen_name, weight] + [pl_stat['std'].loc[f'std_{ten}'] for ten in tenors] + [pl_stat['sharpe'].loc[f'sharpe_{ten}'] for ten in tenors]
    data_list.append(data)
    
data_df = pd.DataFrame(data_list, columns=['strat_name', 'weight'] + [f'std_{ten}' for ten in tenors] + [f'sharpe_{ten}' for ten in tenors])
data_df = data_df.set_index('strat_name')
data_df

In [ ]:
data_df.to_excel("port_perf_check.xlsx")

# asset level cum pnl

In [ ]:
#plot_stats = scen_stats[0]
plot_stats = pnl_stats
asset_pnl = plot_stats['asset_cumpnl']
asset_stats = plot_stats['asset_sharpe_stats']
plot_start = pd.to_datetime('2020-01-01')

rows = math.ceil(len(product_list)/2)
fig, ax = plt.subplots(rows, 2, figsize=(16, 150))

for i, col in enumerate(asset_pnl.columns):
    row_id = i//2
    col_id = i % 2
    asset_pnl[col][plot_start:].plot(ax = ax[row_id, col_id], title = col)

print(asset_stats[product_list])
    

# smoothed pnl and lagged pnl

In [ ]:

bt_metrics = scen_metrics[2]
smoothed = bt_metrics.smoothed_pnl(smooth_hls=[1, 5, 10, 20, 30, 60, 75, 80])
smoothed['cumpnl'].plot(figsize=(10,10))
print('smoothed PNL\n', smoothed['sharpe'])
plt.show()

lagged = bt_metrics.lagged_pnl(lags=[1, 5, 10, 20, 30, 60, 75, 80])
lagged['cumpnl'].plot()
print('lagged PNL\n', lagged['sharpe'])
plt.show()

# scenario PNL and correlations

In [ ]:
scen_start = pd.to_datetime('2014-01-01')
df_list = []
hot_key = 'hot_test'
for scen_name, scen_stat in zip(scen_names[hot_key], scen_stats[hot_key]):
    scen_stat['portfolio_cumpnl'][scen_start:].plot(title=scen_name)
    plt.show()
    ts = scen_stat['portfolio_pnl'][scen_start:]
    ts.name = scen_name
    df_list.append(ts)
dpnl_df = pd.concat(df_list, axis=1, join='outer').fillna(0)
dpnl_df.columns = scen_names[hot_key]
scen_corr = dpnl_df.corr()

# print(scen_corr)
scen_corr.to_csv(file_folder + "port_corr.csv")

# some test for backtest code

In [ ]:
input_args = run_args
product_list = input_args['product_list']
vol_win = input_args['std_win']
total_risk = input_args.get('total_risk', 5000000.0)
shift_mode = input_args.get('shift_mode', 1)
asset_scaling = input_args.get('asset_scaling', False)
exec_mode = input_args.get('exec_mode', 'open')
signal_df = generate_signal(df, input_args)

start_date = input_args.get('start_date', None)
end_date = input_args.get('end_date', None)

if start_date:
    signal_df = signal_df[signal_df.index >= pd.to_datetime(start_date)]
if end_date:
    signal_df = signal_df[signal_df.index <= pd.to_datetime(end_date)]

if shift_mode == 1:
    vol_df = get_asset_vols(df, product_list, vol_win=vol_win, vol_type='atr')
elif shift_mode == 2:
    vol_df = get_asset_vols(df, product_list, vol_win=vol_win, vol_type='lret')
else:
    vol_df = get_asset_vols(df, product_list, vol_win=vol_win, vol_type='close')


In [ ]:
holding = generate_holding_from_signal(signal_df, vol_df,
                                       risk_scaling=total_risk,
                                       asset_scaling=asset_scaling)
holding

In [ ]:
mpl.rcParams['figure.figsize'] = (8, 6)

# batch run backtest for multiple factors

In [ ]:
from bktest.backtest_grid_search import *

In [ ]:
df, error_list = load_hist_data(
    start_date=datetime.date(2010,1,1),
    end_date=datetime.date(2022,1,20),
    sim_markets=commod_all_mkts,
    freq='d'
)

if len(error_list) > 0:
    print(error_list)
print(df.tail(10))

In [ ]:
start_d = datetime.date(2012,1,1)
end_d = datetime.date(2020,1,1)
product_list = ['rb', 'hc', 'i', 'j', 'jm', 'ru', 'FG', 'cu', 'al', 'zn', 'pb', 'sn', \
                'l', 'pp', 'v', 'TA', 'sc', 'm', 'RM', 'y', 'p', 'OI', 'a', 'c', 'CF', 'jd', \
                'AP', 'SM', 'eb', 'eg', 'UR', 'ss', 'lu', 'lh', 'ni',]

sim_group = [
    ('xscarry-rank', 'basmomema'), ('xscarry-rank_cutoff', 'basmomema'), ('xscarry-demedian', 'basmomema'),
    ('xscarry-rank', 'basmomqtl'), ('xscarry-rank_cutoff', 'basmomqtl'), ('xscarry-demedian', 'basmomqtl'),
]

index_list = range(10, 260, 10)
column_list = [10, 20, 40, 61, 80, 100, 122, 244]

bt_metric_dict = {}
pnl_stats_dict = {}
for sim_type, signal_name in sim_group:
    print(f"processing {sim_type} - {signal_name}")
    metric_dict, stat_dict = run_grid_btest(df, start_d, end_d, 
                                            sim_type, signal_name,
                                            index_list=index_list,
                                            column_list=column_list,
                                            product_list=product_list,
                                            pnl_tenors=True,
                                            exp_mean=False)
    bt_metric_dict[(sim_type, signal_name)] = metric_dict
    pnl_stats_dict[(sim_type, signal_name)] = stat_dict


# Batch run sample code 2

In [ ]:
from bktest.backtest_grid_search import *

xdf, error_list = load_hist_data(
    start_date=datetime.date(2010, 1, 1),
    end_date=datetime.date(2020, 1, 1),
    sim_markets=all_markets,
    freq='d'
)


group_keys=['all']

sim_group=[
    ('xsmom-demedian', 'momnma'),
    ('xsmom-demedian', 'macdnma'),
    # ('xscarry-rank', 'ryieldnma'),
    # ('xscarry-rank_cutoff', 'ryieldnma'),
    # ('xscarry-rank', 'basmomnma'),
    # ('xscarry-rank_cutoff', 'basmomnma'),
    # ('xscarry-rank', 'ryieldsma'),
    # ('xscarry-rank_cutoff', 'ryieldsma'),
]

bt_metric_dict, pnl_stats_dict = run_xs_product(xdf, group_keys, sim_group)
